In [1]:
import pandas as pd
import numpy as np
import csv

In [8]:
c_atoms = ['N1','C2','N3','C4','C5','C6','C8','N7','N9']
data = pd.read_csv('stacking_pure.csv')
data.head()

,Molecule_ID,Atom_Name,Atom_ID,x,y,z,time
0,1ADE,O5',1,3.764,2.397,5.202,0.0
1,1ADE,H5T,2,3.701,2.466,5.181,0.0
2,1ADE,C5',3,3.693,2.286,5.258,0.0
3,1ADE,H5',4,3.614,2.332,5.322,0.0
4,1ADE,H5'',5,3.754,2.218,5.320,0.0


#### gro to csv

In [2]:
import csv
import pandas as pd

In [ ]:
# # Open the input GRO file and output CSV file
# with open('/content/md.gro', 'r') as infile, open('output.csv', 'w', newline='') as outfile:
#     # Initialize a CSV writer
#     csv_writer = csv.writer(outfile)
#     # Write the header including the new column 't'
#     csv_writer.writerow(['Molecule_ID', 'Atom_Name', 'Atom_ID', 'x', 'y', 'z', 'time'])

#     # Initialize the time value
#     time_value = None

#     for line in infile:
#         if line[:3] == "Pro":
#             time_value = float(line[19:30].strip())
#         elif line.startswith('49233'):
#             continue
#         elif line[5:9] =="ADE":
#             # Extract relevant information from the GRO file format
#             molecule_id = line[0:5].strip()
#             atom_name = line[11:15]
#             atom_id = line[15:20].strip()
#             x = float(line[20:28])
#             y = float(line[28:36])
#             z = float(line[36:44])
#             # Write the data into the CSV file, including the extracted time value
#             csv_writer.writerow([molecule_id, atom_name, atom_id, x, y, z, time_value])


In [7]:
# Open the input GRO file and output CSV file
with open('pure\stacking_short.gro', 'r') as infile, open('stacking_pure.csv', 'w', newline='') as outfile:
    # Initialize a CSV writer
    csv_writer = csv.writer(outfile)
    # Write the header including the new column 'time'
    csv_writer.writerow(['Molecule_ID', 'Atom_Name', 'Atom_ID', 'x', 'y', 'z', 'time'])

    # Initialize the time value
    time_value = None

    for line in infile:
        if line.startswith('Pro'):
            time_value = float(line[19:30].strip())
        elif 'ADE' in line:
            # Split the line into entries
            entries = line.split()
            # Extract relevant information from the GRO file format
            molecule_id = entries[0]
            atom_name = entries[1]
            atom_id = entries[2]
            x = float(entries[3])
            y = float(entries[4])
            z = float(entries[5])
            # Write the data into the CSV file, including the extracted time value
            csv_writer.writerow([molecule_id, atom_name, atom_id, x, y, z, time_value])
            # print("Writing data:", [molecule_id, atom_name, atom_id, x, y, z, time_value])


#### CSV file

,Molecule_ID,Atom_Name,Atom_ID,x,y,z,time,c_x,c_y,c_z
0,1ADE,O5',1,6.666,4.157,1.655,0.0,6.573444,3.645,1.965222
1,1ADE,H5T,2,6.668,4.215,1.578,0.0,6.573444,3.645,1.965222
2,1ADE,C5',3,6.603,4.232,1.758,0.0,6.573444,3.645,1.965222
3,1ADE,H5',4,6.499,4.247,1.722,0.0,6.573444,3.645,1.965222
4,1ADE,H5'',5,6.649,4.333,1.768,0.0,6.573444,3.645,1.965222


In [ ]:
# data['time'].value_counts()

In [6]:
data['Atom_Name'].unique()

array(["O5'", 'H5T', "C5'", "H5'", "H5''", "C4'", "H4'", "O4'", "C1'",
       "H1'", 'N9', 'C5', 'N7', 'C8', 'H8', 'N1', 'C2', 'H2', 'N3', 'C4',
       'C6', 'N6', 'H61', 'H62', "C2'", "H2''", "O2'", "H2'", "C3'",
       "H3'", "O3'", 'P', 'O1P', 'O2P', 'H3T'], dtype=object)

In [9]:
time = 995000.0
data.dtypes

Molecule_ID     object
Atom_Name       object
Atom_ID          int64
x              float64
y              float64
z              float64
time           float64
dtype: object

In [10]:
data['c_x'] = 0
data['c_y'] = 0
data['c_z'] = 0

In [11]:
# Function to calculate center of mass for a given residue
def calculate_center_of_mass(residue_data,lst):
    # Filter data for the atoms in lst
    filtered_data = residue_data[residue_data['Atom_Name'].isin(lst)]

    # Calculate center of mass for x, y, and z axes
    total_mass = len(filtered_data)
    if total_mass == 0:
        return None, None, None
    else:
        center_of_mass_x = filtered_data['x'].mean()
        center_of_mass_y = filtered_data['y'].mean()
        center_of_mass_z = filtered_data['z'].mean()
        return center_of_mass_x, center_of_mass_y, center_of_mass_z

In [12]:
# Calculate center of mass for each residue at each time point
for (time, residue_id), residue_data in data.groupby(['time', 'Molecule_ID']):
    center_of_mass_x, center_of_mass_y, center_of_mass_z = calculate_center_of_mass(residue_data,c_atoms)
    data.loc[(data['time'] == time) & (data['Molecule_ID'] == residue_id), 'c_x'] = center_of_mass_x
    data.loc[(data['time'] == time) & (data['Molecule_ID'] == residue_id), 'c_y'] = center_of_mass_y
    data.loc[(data['time'] == time) & (data['Molecule_ID'] == residue_id), 'c_z'] = center_of_mass_z

data.head(15)

C:\Users\Bojja\AppData\Local\Temp\ipykernel_352\2821273459.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.576888888888889' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[(data['time'] == time) & (data['Molecule_ID'] == residue_id), 'c_x'] = center_of_mass_x
C:\Users\Bojja\AppData\Local\Temp\ipykernel_352\2821273459.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.788555555555556' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[(data['time'] == time) & (data['Molecule_ID'] == residue_id), 'c_y'] = center_of_mass_y
C:\Users\Bojja\AppData\Local\Temp\ipykernel_352\2821273459.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.968888

,Molecule_ID,Atom_Name,Atom_ID,x,y,z,time,c_x,c_y,c_z
0,1ADE,O5',1,3.764,2.397,5.202,0.0,3.419,2.495556,4.784667
1,1ADE,H5T,2,3.701,2.466,5.181,0.0,3.419,2.495556,4.784667
2,1ADE,C5',3,3.693,2.286,5.258,0.0,3.419,2.495556,4.784667
3,1ADE,H5',4,3.614,2.332,5.322,0.0,3.419,2.495556,4.784667
4,1ADE,H5'',5,3.754,2.218,5.320,0.0,3.419,2.495556,4.784667
5,1ADE,C4',6,3.621,2.202,5.153,0.0,3.419,2.495556,4.784667
6,1ADE,H4',7,3.578,2.115,5.208,0.0,3.419,2.495556,4.784667
7,1ADE,O4',8,3.504,2.268,5.094,0.0,3.419,2.495556,4.784667
8,1ADE,C1',9,3.506,2.243,4.947,0.0,3.419,2.495556,4.784667
9,1ADE,H1',10,3.404,2.202,4.932,0.0,3.419,2.495556,4.784667


In [13]:
data.to_csv('stacking_pure.csv', index=False)


#### Stacking

#### Condition 1

In [ ]:
time_point = 995000.0
time_data = data[data['time'] == time_point]
unique_residue_ids = time_data['Molecule_ID'].unique()

In [ ]:
# Calculate center of mass for each residue and store in a dictionary
center_of_mass_dict = {}
for residue_id in unique_residue_ids:
    residue_data = time_data[time_data['Molecule_ID'] == residue_id]
    c_x = residue_data['c_x'].values[0] if not residue_data.empty else None
    c_y = residue_data['c_y'].values[0] if not residue_data.empty else None
    c_z = residue_data['c_z'].values[0] if not residue_data.empty else None
    center_of_mass_dict[residue_id] = (c_x, c_y, c_z)


{'1ADE': (4.663777777777778, 1.776111111111111, 4.293222222222223), '2ADE': (4.630888888888889, 2.240222222222222, 4.971888888888889), '3ADE': (4.837444444444444, 2.0564444444444443, 4.531777777777778), '4ADE': (4.393555555555555, 1.7418888888888888, 3.97), '5ADE': (4.440555555555556, 2.706111111111111, 4.026666666666667), '6ADE': (4.299666666666666, 3.2432222222222222, 3.836777777777778), '7ADE': (4.774, 3.687777777777778, 3.584888888888889), '8ADE': (4.931333333333333, 3.847555555555556, 3.974222222222222), '9ADE': (5.001, 4.050777777777777, 4.484555555555556), '10ADE': (4.674, 4.435444444444444, 4.492555555555556), '11ADE': (3.647666666666667, 4.426777777777777, 3.909444444444445), '12ADE': (3.660444444444445, 3.870444444444444, 3.629333333333333), '13ADE': (3.870333333333333, 3.3695555555555554, 3.398777777777778), '14ADE': (3.5460000000000003, 3.879555555555556, 3.2598888888888893), '15ADE': (3.924666666666666, 4.689, 3.838333333333333), '16ADE': (4.1803333333333335, 5.164, 3.9178

In [ ]:
# Initialize distance matrix
num_residues = len(unique_residue_ids)
dist_matrix = np.zeros((num_residues, num_residues))

# Calculate distances between the center of mass of different residues and fill the distance matrix
for i in range(num_residues):
    for j in range(i + 1, num_residues):
        residue_id_1 = unique_residue_ids[i]
        residue_id_2 = unique_residue_ids[j]
        center_of_mass_1 = np.array(center_of_mass_dict[residue_id_1])
        center_of_mass_2 = np.array(center_of_mass_dict[residue_id_2])
        distance = np.linalg.norm(center_of_mass_1 - center_of_mass_2)
        dist_matrix[i, j] = distance
        # dist_matrix[j, i] = distance  # Distance matrix is symmetric


used a gro file so the values will be in nm isntead of ang

In [ ]:
# Display the distance matrix
print("Distance Matrix:")
print(dist_matrix[0])

Distance Matrix:
[0.         0.82284218 0.40700816 0.42268643 0.99286456 1.57902927
 2.04165484 2.11287223 2.30747388 2.66681309 2.86457692 2.41529674
 1.99214712 2.5964752  3.03942936 3.44272926 3.07701429 2.87628084
 2.52321369 2.9795869 ]


In [ ]:
cnt = 0
pairs = []
for i in range(len(dist_matrix)):
    for j in range(len(dist_matrix[i])):
        if dist_matrix[i][j]<=0.55 and dist_matrix[i][j]>0:
            cnt+=1
            pairs.append(list((str(i+1)+'ADE',str(j+1)+'ADE')))
print(cnt)
print(pairs)
unique_elements = set()
for a, b in pairs:
    unique_elements.add(a)
    unique_elements.add(b)
lst = list(unique_elements)
print("lst", lst)



15
[['1ADE', '3ADE'], ['1ADE', '4ADE'], ['2ADE', '3ADE'], ['7ADE', '8ADE'], ['8ADE', '19ADE'], ['9ADE', '10ADE'], ['10ADE', '18ADE'], ['11ADE', '15ADE'], ['12ADE', '14ADE'], ['15ADE', '16ADE'], ['15ADE', '17ADE'], ['16ADE', '17ADE'], ['17ADE', '20ADE'], ['18ADE', '19ADE'], ['19ADE', '20ADE']]
lst ['9ADE', '3ADE', '15ADE', '17ADE', '11ADE', '16ADE', '20ADE', '4ADE', '1ADE', '10ADE', '12ADE', '8ADE', '14ADE', '7ADE', '2ADE', '19ADE', '18ADE']


In [ ]:
dist_matrix[0]

array([0.        , 0.82284218, 0.40700816, 0.42268643, 0.99286456,
       1.57902927, 2.04165484, 2.11287223, 2.30747388, 2.66681309,
       2.86457692, 2.41529674, 1.99214712, 2.5964752 , 3.03942936,
       3.44272926, 3.07701429, 2.87628084, 2.52321369, 2.9795869 ])

#### Condition 2

finding normal vectors

In [ ]:
c_atoms[:2]

['N1', 'C2']

1. R1 = N1.Centroid vector
2. R2 = C2.centroid vector
3. Normal = R1 X R2 unit vector

In [ ]:
data.columns

Index(['Molecule_ID', 'Atom_Name', 'Atom_ID', 'x', 'y', 'z', 'time', 'c_x',
       'c_y', 'c_z'],
      dtype='object')

In [ ]:
#create a new dataframe with only the pairs as molecule id and N1,C2 as atom names
df  = time_data[time_data['Atom_Name'].isin(c_atoms[:2]) & time_data['Molecule_ID'].isin(lst)]
df.head()

,Molecule_ID,Atom_Name,Atom_ID,x,y,z,time,c_x,c_y,c_z
131156,1ADE,N1,16,4.659,1.975,4.195,995000.0,4.663778,1.776111,4.293222
131157,1ADE,C2,17,4.544,1.943,4.239,995000.0,4.663778,1.776111,4.293222
131189,2ADE,N1,49,4.711,2.443,4.948,995000.0,4.630889,2.240222,4.971889
131190,2ADE,C2,50,4.596,2.443,5.023,995000.0,4.630889,2.240222,4.971889
131222,3ADE,N1,82,5.050,2.061,4.485,995000.0,4.837444,2.056444,4.531778


In [ ]:
df['nx'] = np.nan
df['ny'] = np.nan
df['nz'] = np.nan

In [ ]:
# Iterate over unique residue IDs and calculate the normal unit vector
for residue_id in df['Molecule_ID'].unique():
    residue_data = df[df['Molecule_ID'] == residue_id]
    # Convert columns to numeric type if necessary
    numeric_columns = ['x', 'y', 'z', 'c_x', 'c_y', 'c_z']
    residue_data[numeric_columns] = residue_data[numeric_columns].apply(pd.to_numeric)

    # Now calculate the cross product
    n1 = residue_data.iloc[0][['x', 'y', 'z', 'c_x', 'c_y', 'c_z']].values[:3].astype(float)
    cen = residue_data.iloc[0][['x', 'y', 'z', 'c_x', 'c_y', 'c_z']].values[3:].astype(float)
    c2 = residue_data.iloc[1][['x', 'y', 'z', 'c_x', 'c_y', 'c_z']].values[:3].astype(float)
    r1 = n1 - cen
    r2 = c2 - cen
    v = np.cross(r1, r2)  # Calculate cross product
    unit_v = v / np.linalg.norm(v)  # Normalize to obtain unit vector
    df.loc[df['Molecule_ID'] == residue_id, ['nx', 'ny', 'nz']] = unit_v



In [ ]:
df.head()

,Molecule_ID,Atom_Name,Atom_ID,x,y,z,time,c_x,c_y,c_z,nx,ny,nz
131156,1ADE,N1,16,4.659,1.975,4.195,995000.0,4.663778,1.776111,4.293222,0.212879,0.436757,0.874029
131157,1ADE,C2,17,4.544,1.943,4.239,995000.0,4.663778,1.776111,4.293222,0.212879,0.436757,0.874029
131189,2ADE,N1,49,4.711,2.443,4.948,995000.0,4.630889,2.240222,4.971889,0.542558,-0.116340,0.831923
131190,2ADE,C2,50,4.596,2.443,5.023,995000.0,4.630889,2.240222,4.971889,0.542558,-0.116340,0.831923
131222,3ADE,N1,82,5.050,2.061,4.485,995000.0,4.837444,2.056444,4.531778,0.205782,0.210905,0.955601


In [ ]:
print("Pairs",pairs)
print("lst",lst)

Pairs [['1ADE', '3ADE'], ['1ADE', '4ADE'], ['2ADE', '3ADE'], ['7ADE', '8ADE'], ['8ADE', '19ADE'], ['9ADE', '10ADE'], ['10ADE', '18ADE'], ['11ADE', '15ADE'], ['12ADE', '14ADE'], ['15ADE', '16ADE'], ['15ADE', '17ADE'], ['16ADE', '17ADE'], ['17ADE', '20ADE'], ['18ADE', '19ADE'], ['19ADE', '20ADE']]
lst ['9ADE', '3ADE', '15ADE', '17ADE', '11ADE', '16ADE', '20ADE', '4ADE', '1ADE', '10ADE', '12ADE', '8ADE', '14ADE', '7ADE', '2ADE', '19ADE', '18ADE']


In [ ]:
# Iterate over pairs
for pair in pairs:
    res1, res2 = pair[0], pair[1]

    # Get data for each residue
    residue_data1 = df[df['Molecule_ID'] == res1]
    residue_data2 = df[df['Molecule_ID'] == res2]

    # Check if residue data is available
    if len(residue_data1) == 0 or len(residue_data2) == 0:
        continue

    # Convert columns to numeric type if necessary
    numeric_columns = ['nx', 'ny', 'nz']
    residue_data1[numeric_columns] = residue_data1[numeric_columns].apply(pd.to_numeric)
    residue_data2[numeric_columns] = residue_data2[numeric_columns].apply(pd.to_numeric)

    # Calculate angle between n1 and n2
    n1 = residue_data1.iloc[0][numeric_columns].values
    n2 = residue_data2.iloc[0][numeric_columns].values
    angle = np.arccos(np.dot(n1, n2) / (np.linalg.norm(n1) * np.linalg.norm(n2)))

    # Convert angle to degrees if needed
    angle_degrees = np.degrees(angle)
    pair.append(angle_degrees)
    # Append angle to the pair or do further processing as needed
    print("Angle between", res1, "and", res2, ":", angle_degrees)


In [ ]:
pairs

[['1ADE', '3ADE', 13.797829404160028],
 ['1ADE', '4ADE', 18.02776184974261],
 ['2ADE', '3ADE', 28.10353018254178],
 ['7ADE', '8ADE', 11.356941403874545],
 ['8ADE', '19ADE', 176.85423620684475],
 ['9ADE', '10ADE', 51.243625083328006],
 ['10ADE', '18ADE', 171.1913678256133],
 ['11ADE', '15ADE', 5.169380228156999],
 ['12ADE', '14ADE', 15.221049027356305],
 ['15ADE', '16ADE', 23.790996966242243],
 ['15ADE', '17ADE', 46.69586275814026],
 ['16ADE', '17ADE', 24.848166704805493],
 ['17ADE', '20ADE', 123.3327863572365],
 ['18ADE', '19ADE', 75.89588443067866],
 ['19ADE', '20ADE', 157.28627322495328]]

In [ ]:
pairs = [pair for pair in pairs if pair[2] <= 30]
pairs

[['1ADE', '3ADE', 13.797829404160028],
 ['1ADE', '4ADE', 18.02776184974261],
 ['2ADE', '3ADE', 28.10353018254178],
 ['7ADE', '8ADE', 11.356941403874545],
 ['11ADE', '15ADE', 5.169380228156999],
 ['12ADE', '14ADE', 15.221049027356305],
 ['15ADE', '16ADE', 23.790996966242243],
 ['16ADE', '17ADE', 24.848166704805493]]

In [ ]:
len(pairs)

8

#### Condition 3

In [ ]:
df.head()

,Molecule_ID,Atom_Name,Atom_ID,x,y,z,time,c_x,c_y,c_z,nx,ny,nz
131156,1ADE,N1,16,4.659,1.975,4.195,995000.0,4.663778,1.776111,4.293222,0.212879,0.436757,0.874029
131157,1ADE,C2,17,4.544,1.943,4.239,995000.0,4.663778,1.776111,4.293222,0.212879,0.436757,0.874029
131189,2ADE,N1,49,4.711,2.443,4.948,995000.0,4.630889,2.240222,4.971889,0.542558,-0.116340,0.831923
131190,2ADE,C2,50,4.596,2.443,5.023,995000.0,4.630889,2.240222,4.971889,0.542558,-0.116340,0.831923
131222,3ADE,N1,82,5.050,2.061,4.485,995000.0,4.837444,2.056444,4.531778,0.205782,0.210905,0.955601


In [ ]:
pairs

[['1ADE', '3ADE', 13.797829404160028],
 ['1ADE', '4ADE', 18.02776184974261],
 ['2ADE', '3ADE', 28.10353018254178],
 ['7ADE', '8ADE', 11.356941403874545],
 ['11ADE', '15ADE', 5.169380228156999],
 ['12ADE', '14ADE', 15.221049027356305],
 ['15ADE', '16ADE', 23.790996966242243],
 ['16ADE', '17ADE', 24.848166704805493]]

In [ ]:
# Iterate over pairs
for pair in pairs:
    res1, res2= pair[0], pair[1]

    # Get data for each residue
    residue_data1 = df[df['Molecule_ID'] == res1]
    residue_data2 = df[df['Molecule_ID'] == res2]

    # Check if residue data is available
    if len(residue_data1) == 0 or len(residue_data2) == 0:
        continue

    # Convert columns to numeric type if necessary
    normal_columns = ['nx', 'ny', 'nz']
    centroid_columns = ['c_x', 'c_y', 'c_z']
    residue_data1[normal_columns] = residue_data1[normal_columns].apply(pd.to_numeric)
    residue_data1[centroid_columns] = residue_data1[centroid_columns].apply(pd.to_numeric)
    residue_data2[centroid_columns] = residue_data2[centroid_columns].apply(pd.to_numeric)

    # Calculate angle between n1 and n2
    n1 = residue_data1.iloc[0][normal_columns].values
    c1 = residue_data1.iloc[0][centroid_columns].values
    c2 = residue_data2.iloc[0][centroid_columns].values
    d12 = c2-c1
    angle = np.arccos(np.dot(n1, d12) / (np.linalg.norm(n1) * np.linalg.norm(d12)))

    # Convert angle to degrees if needed
    angle_degrees = np.degrees(angle)
    pair.append(angle_degrees)
    # Append angle to the pair or do further processing as needed
    print("Angle between", res1, "and", res2, ":", angle_degrees)


In [ ]:
pairs = [pair for pair in pairs if pair[3] <= 40]
print(len(pairs))
pairs

2


[['1ADE', '3ADE', 13.797829404160028, 25.31881240903482],
 ['12ADE', '14ADE', 15.221049027356305, 22.48502604493388]]

#### Functions

In [ ]:
def stacking(df,t_value):
    # Condition 1: Dist matrix
    # Calculate centroids
    time_data = df[df['time'] == t_value]
    unique_residue_ids = time_data['Molecule_ID'].unique()
    center_of_mass_dict = {}
    for residue_id in unique_residue_ids:
        residue_data = time_data[time_data['Molecule_ID'] == residue_id]
        c_x = residue_data['c_x'].values[0] if not residue_data.empty else None
        c_y = residue_data['c_y'].values[0] if not residue_data.empty else None
        c_z = residue_data['c_z'].values[0] if not residue_data.empty else None
        center_of_mass_dict[residue_id] = (c_x, c_y, c_z)

    # Initialize distance matrix
    num_residues = len(unique_residue_ids)
    dist_matrix = np.zeros((num_residues, num_residues))

    # Calculate distances between the center of mass of different residues and fill the distance matrix
    for i in range(num_residues):
        for j in range(i + 1, num_residues):
            residue_id_1 = unique_residue_ids[i]
            residue_id_2 = unique_residue_ids[j]
            center_of_mass_1 = np.array(center_of_mass_dict[residue_id_1])
            center_of_mass_2 = np.array(center_of_mass_dict[residue_id_2])
            distance = np.linalg.norm(center_of_mass_1 - center_of_mass_2)
            dist_matrix[i, j] = distance

    pairs = []
    for i in range(len(dist_matrix)):
        for j in range(len(dist_matrix[i])):
            if dist_matrix[i][j] <= 0.55 and dist_matrix[i][j] > 0:
                pairs.append(list((str(i+1) + 'ADE', str(j+1) + 'ADE')))

    # Condition 2: Atom Name and Molecule ID filtering
    df_filtered = time_data[time_data['Atom_Name'].isin(c_atoms[:2]) & time_data['Molecule_ID'].isin(lst)]
    df_filtered['nx'] = np.nan
    df_filtered['ny'] = np.nan
    df_filtered['nz'] = np.nan

    # Calculate normal unit vectors for each residue
    for residue_id in df_filtered['Molecule_ID'].unique():
        residue_data = df_filtered[df_filtered['Molecule_ID'] == residue_id]
        # Convert columns to numeric type if necessary
        numeric_columns = ['x', 'y', 'z', 'c_x', 'c_y', 'c_z']
        residue_data[numeric_columns] = residue_data[numeric_columns].apply(pd.to_numeric)

        # Now calculate the cross product
        n1 = residue_data.iloc[0][['x', 'y', 'z', 'c_x', 'c_y', 'c_z']].values[:3].astype(float)
        cen = residue_data.iloc[0][['x', 'y', 'z', 'c_x', 'c_y', 'c_z']].values[3:].astype(float)
        c2 = residue_data.iloc[1][['x', 'y', 'z', 'c_x', 'c_y', 'c_z']].values[:3].astype(float)
        r1 = n1 - cen
        r2 = c2 - cen
        v = np.cross(r1, r2)  # Calculate cross product
        unit_v = v / np.linalg.norm(v)  # Normalize to obtain unit vector
        df_filtered.loc[df_filtered['Molecule_ID'] == residue_id, ['nx', 'ny', 'nz']] = unit_v

    # Iterate over pairs
    for pair in pairs:
        res1, res2 = pair[0], pair[1]

        # Get data for each residue
        residue_data1 = df_filtered[df_filtered['Molecule_ID'] == res1]
        residue_data2 = df_filtered[df_filtered['Molecule_ID'] == res2]

        # Check if residue data is available
        if len(residue_data1) == 0 or len(residue_data2) == 0:
            continue

        # Convert columns to numeric type if necessary
        numeric_columns = ['nx', 'ny', 'nz']
        residue_data1[numeric_columns] = residue_data1[numeric_columns].apply(pd.to_numeric)
        residue_data2[numeric_columns] = residue_data2[numeric_columns].apply(pd.to_numeric)

        # Calculate angle between n1 and n2
        n1 = residue_data1.iloc[0][numeric_columns].values
        n2 = residue_data2.iloc[0][numeric_columns].values
        angle = np.arccos(np.dot(n1, n2) / (np.linalg.norm(n1) * np.linalg.norm(n2)))

        # Convert angle to degrees if needed
        angle_degrees = np.degrees(angle)
        pair.append(angle_degrees)
    if len(pairs)==0:
        return 0
    pairs = [pair for pair in pairs if len(pair) >= 3 and pair[2] <= 30]

    # Condition 3
    for pair in pairs:
        res1, res2= pair[0], pair[1]

        # Get data for each residue
        residue_data1 = df_filtered[df_filtered['Molecule_ID'] == res1]
        residue_data2 = df_filtered[df_filtered['Molecule_ID'] == res2]

        # Check if residue data is available
        if len(residue_data1) == 0 or len(residue_data2) == 0:
            continue

        # Convert columns to numeric type if necessary
        normal_columns = ['nx', 'ny', 'nz']
        centroid_columns = ['c_x', 'c_y', 'c_z']
        residue_data1[normal_columns] = residue_data1[normal_columns].apply(pd.to_numeric)
        residue_data1[centroid_columns] = residue_data1[centroid_columns].apply(pd.to_numeric)
        residue_data2[centroid_columns] = residue_data2[centroid_columns].apply(pd.to_numeric)

        # Calculate angle between n1 and n2
        n1 = residue_data1.iloc[0][normal_columns].values
        c1 = residue_data1.iloc[0][centroid_columns].values
        c2 = residue_data2.iloc[0][centroid_columns].values
        d12 = c2 - c1
        angle = np.arccos(np.dot(n1, d12) / (np.linalg.norm(n1) * np.linalg.norm(d12)))

        # Convert angle to degrees if needed
        angle_degrees = np.degrees(angle)
        pair.append(angle_degrees)
    if len(pairs)==0:
        return 0
    pairs = [pair for pair in pairs if len(pair) >= 4 and pair[3] <= 40]

    return len(pairs)


In [ ]:
data.columns

Index(['Molecule_ID', 'Atom_Name', 'Atom_ID', 'x', 'y', 'z', 'time', 'c_x',
       'c_y', 'c_z'],
      dtype='object')

In [ ]:
lst = []
for time_value in data['time']:
    lst.append(stacking(data,time_value))
print(lst)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 